In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import pandas as pd
from datasets import load_dataset, Dataset, Audio
from tqdm import trange
import numpy as np
from src.audio_mixer import mixer
from transformers import WhisperFeatureExtractor,WhisperTokenizer,WhisperProcessor
from sklearn.model_selection import train_test_split
from src.data_engineering import creating_random_split_df



## Preparing data

#### Testing batch

In [3]:
df_test = pd.read_parquet('./data/parquets/testing_batch.parquet.gzip')
df_test

,audioname,dataset,ref_orig,sampling_rate,audiopath_bigos,audiopath_local
0,fair-mls-20-train-0009-04739,fair-mls-20,tam nocne włóczęgi wołano z dachów jeżeli nie ...,16000,fair-mls-20-train-0009-04739.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
1,pjatk-clarin_studio-15-train-0488-00003,pjatk-clarin_studio-15,w pracy studenci chcieliby przede wszystk...,16000,pjatk-clarin_studio-15-train-0488-00003.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
2,fair-mls-20-train-0009-05501,fair-mls-20,co to znaczy sam siebie zapytywał faraon czy g...,16000,fair-mls-20-train-0009-05501.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
3,fair-mls-20-train-0021-01519,fair-mls-20,tylko na piaszczystem wybrzeżu lub na łąkach b...,16000,fair-mls-20-train-0021-01519.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
4,pjatk-clarin_studio-15-train-0335-00001,pjatk-clarin_studio-15,zaokrągla uziemienie księdzu liźnięcie rol...,16000,pjatk-clarin_studio-15-train-0335-00001.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
...,...,...,...,...,...,...
2495,fair-mls-20-train-0009-06517,fair-mls-20,kazał zrobić spis wszystkich mężczyzn w państw...,16000,fair-mls-20-train-0009-06517.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
2496,mozilla-common_voice_15-23-train-2851-00218,mozilla-common_voice_15-23,"W odniesieniu do Lizbony, uczyniliśmy także po...",16000,mozilla-common_voice_15-23-train-2851-00218.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
2497,mozilla-common_voice_15-23-train-2856-01361,mozilla-common_voice_15-23,"Jej budżet to budżet, który wspiera inwestycje",16000,mozilla-common_voice_15-23-train-2856-01361.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
2498,fair-mls-20-train-0009-03165,fair-mls-20,upłynęło już kilka godzin po zachodzie słońca ...,16000,fair-mls-20-train-0009-03165.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...


#### Training batch

In [7]:
ds_full= load_dataset("amu-cai/pl-asr-bigos-v2",'all', 'all')

c:\Users\Eryk\anaconda3\envs\Magisterka\Lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for amu-cai/pl-asr-bigos-v2 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/amu-cai/pl-asr-bigos-v2
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating test split: 0 examples [00:00, ? examples/s]

Generating  examples


DatasetGenerationError: An error occurred while generating the dataset

In [5]:
df_train = pd.DataFrame(ds_full['train'])

In [7]:
del ds_full

In [6]:
df_train = df_train[['audioname','ref_orig','audiopath_bigos','audiopath_local']]
df_train

,audioname,ref_orig,audiopath_bigos,audiopath_local
0,pjatk-clarin_mobile-15-train-0035-00001,proszę państwa senatorów o zajęcie miejsc przy...,pjatk-clarin_mobile-15-train-0035-00001.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
1,pjatk-clarin_mobile-15-train-0035-00002,proszę nowo wybranego senatora stanisława husk...,pjatk-clarin_mobile-15-train-0035-00002.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
2,pjatk-clarin_mobile-15-train-0035-00003,sejm przyjął większość poprawek senatu do usta...,pjatk-clarin_mobile-15-train-0035-00003.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
3,pjatk-clarin_mobile-15-train-0035-00004,pomimo faktu że w tęczy występuje niemal ciągł...,pjatk-clarin_mobile-15-train-0035-00004.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
4,pjatk-clarin_mobile-15-train-0035-00005,był autorem pierwszego w historii polskiego op...,pjatk-clarin_mobile-15-train-0035-00005.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
...,...,...,...,...
82020,polyai-minds14-21-train-0001-00458,Czy mogę przelać pieniądze na konto,polyai-minds14-21-train-0001-00458.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
82021,polyai-minds14-21-train-0001-00459,jestem bardzo zatyczki serdecznie Mam problem ...,polyai-minds14-21-train-0001-00459.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
82022,polyai-minds14-21-train-0001-00460,Dzień dobry Chciałbym wpłacić pieniądze na swo...,polyai-minds14-21-train-0001-00460.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
82023,polyai-minds14-21-train-0001-00461,Chcę wpłacić pieniądze na swoje konto w jakiś ...,polyai-minds14-21-train-0001-00461.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...


#### Not involving datafiles from test set

In [23]:
#Finding common values from both dataframes
common_audionames = set(df_test['audioname']).intersection(df_train['audioname'])

# Filter out rows from df_train where 'audioname' matches with df_test
df_train_filterred = df_train[~df_train['audioname'].isin(common_audionames)]


In [25]:
df_train_filterred = df_train_filterred.reset_index()
df_train_filterred

,index,audioname,ref_orig,audiopath_bigos,audiopath_local
0,0,pjatk-clarin_mobile-15-train-0035-00001,proszę państwa senatorów o zajęcie miejsc przy...,pjatk-clarin_mobile-15-train-0035-00001.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
1,1,pjatk-clarin_mobile-15-train-0035-00002,proszę nowo wybranego senatora stanisława husk...,pjatk-clarin_mobile-15-train-0035-00002.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
2,2,pjatk-clarin_mobile-15-train-0035-00003,sejm przyjął większość poprawek senatu do usta...,pjatk-clarin_mobile-15-train-0035-00003.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
3,3,pjatk-clarin_mobile-15-train-0035-00004,pomimo faktu że w tęczy występuje niemal ciągł...,pjatk-clarin_mobile-15-train-0035-00004.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
4,4,pjatk-clarin_mobile-15-train-0035-00005,był autorem pierwszego w historii polskiego op...,pjatk-clarin_mobile-15-train-0035-00005.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
...,...,...,...,...,...
79520,82020,polyai-minds14-21-train-0001-00458,Czy mogę przelać pieniądze na konto,polyai-minds14-21-train-0001-00458.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
79521,82021,polyai-minds14-21-train-0001-00459,jestem bardzo zatyczki serdecznie Mam problem ...,polyai-minds14-21-train-0001-00459.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
79522,82022,polyai-minds14-21-train-0001-00460,Dzień dobry Chciałbym wpłacić pieniądze na swo...,polyai-minds14-21-train-0001-00460.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...
79523,82023,polyai-minds14-21-train-0001-00461,Chcę wpłacić pieniądze na swoje konto w jakiś ...,polyai-minds14-21-train-0001-00461.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...


In [26]:
import os
import soundfile as sf
import pyloudnorm as pyln

def audio_normalizer(input_path, output_folder_path,output_file_name, loudness_value):

    # Load audio
    data, rate = sf.read(input_path)

    # Measure the loudness
    meter = pyln.Meter(rate, block_size = 0.4)
    loudness = meter.integrated_loudness(data)

    # Loudness normalize audio to -12 dB LUFS
    loudness_normalized_audio = pyln.normalize.loudness(data, loudness, loudness_value)

    # Specify the output file paths
    loudness_normalized_output_path = os.path.join(output_folder_path, output_file_name)

    # Write normalized audio to files
    sf.write(loudness_normalized_output_path, loudness_normalized_audio, rate)


In [35]:
df_train_filterred['audiopath_normalized'] = './data/mixed_recordings/normalised_recordings/audio_minus_20dB/' +  df_train_filterred['audiopath_bigos']

In [36]:
df_train_filterred

,index,audioname,ref_orig,audiopath_bigos,audiopath_local,audiopath_normalized
0,0,pjatk-clarin_mobile-15-train-0035-00001,proszę państwa senatorów o zajęcie miejsc przy...,pjatk-clarin_mobile-15-train-0035-00001.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...
1,1,pjatk-clarin_mobile-15-train-0035-00002,proszę nowo wybranego senatora stanisława husk...,pjatk-clarin_mobile-15-train-0035-00002.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...
2,2,pjatk-clarin_mobile-15-train-0035-00003,sejm przyjął większość poprawek senatu do usta...,pjatk-clarin_mobile-15-train-0035-00003.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...
3,3,pjatk-clarin_mobile-15-train-0035-00004,pomimo faktu że w tęczy występuje niemal ciągł...,pjatk-clarin_mobile-15-train-0035-00004.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...
4,4,pjatk-clarin_mobile-15-train-0035-00005,był autorem pierwszego w historii polskiego op...,pjatk-clarin_mobile-15-train-0035-00005.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...
...,...,...,...,...,...,...
79520,82020,polyai-minds14-21-train-0001-00458,Czy mogę przelać pieniądze na konto,polyai-minds14-21-train-0001-00458.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...
79521,82021,polyai-minds14-21-train-0001-00459,jestem bardzo zatyczki serdecznie Mam problem ...,polyai-minds14-21-train-0001-00459.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...
79522,82022,polyai-minds14-21-train-0001-00460,Dzień dobry Chciałbym wpłacić pieniądze na swo...,polyai-minds14-21-train-0001-00460.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...
79523,82023,polyai-minds14-21-train-0001-00461,Chcę wpłacić pieniądze na swoje konto w jakiś ...,polyai-minds14-21-train-0001-00461.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...


In [30]:
input_paths = df_train_filterred['audiopath_local'].to_list()
file_names = df_train_filterred['audiopath_bigos'].to_list()
output_folder_path = "./data/mixed_recordings/normalised_recordings/audio_minus_20dB/"
for i in trange(len(input_paths)):
    audio_normalizer(input_paths[i], output_folder_path,file_names[i], -20.0)


  0%|          | 2/79525 [00:00<3:13:29,  6.85it/s]c:\Users\Eryk\anaconda3\envs\Magisterka\Lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
  0%|          | 18/79525 [00:00<28:01, 47.30it/s] c:\Users\Eryk\anaconda3\envs\Magisterka\Lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
  0%|          | 26/79525 [00:00<23:10, 57.16it/s]c:\Users\Eryk\anaconda3\envs\Magisterka\Lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\Eryk\anaconda3\envs\Magisterka\Lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
  0%|          | 33/79525 [00:00<21:59, 60.24it/s]c:\Users\Eryk\anaconda3\envs\Magisterk

In [33]:
df_visc = pd.read_parquet('./data/parquets/VISC_noises.parquet.gzip')
visc_noises_dict = {1 : 'bus_interior',
                    2 : 'minibus_interior',
                    3 : 'pickup_interior',
                    4 : 'sports_car_interior',
                    5 : 'jeep_interior',
                    6 : 'truck_interior',
                    7 : 'crossover_interior',
                    8 : 'other_car_interior'}
df_visc['class'] = df_visc['class_id'].map(visc_noises_dict)
df_visc

,file_names,file_path,class_id,class
0,1 (1).wav,.\data\VISC Dataset SON\1 (1).wav,1,bus_interior
1,1 (10).wav,.\data\VISC Dataset SON\1 (10).wav,1,bus_interior
2,1 (100).wav,.\data\VISC Dataset SON\1 (100).wav,1,bus_interior
3,1 (101).wav,.\data\VISC Dataset SON\1 (101).wav,1,bus_interior
4,1 (102).wav,.\data\VISC Dataset SON\1 (102).wav,1,bus_interior
...,...,...,...,...
5975,8 (95).wav,.\data\VISC Dataset SON\8 (95).wav,8,other_car_interior
5976,8 (96).wav,.\data\VISC Dataset SON\8 (96).wav,8,other_car_interior
5977,8 (97).wav,.\data\VISC Dataset SON\8 (97).wav,8,other_car_interior
5978,8 (98).wav,.\data\VISC Dataset SON\8 (98).wav,8,other_car_interior


In [34]:
input_paths = df_visc['file_path'].to_list()
file_names = df_visc['file_names'].to_list()
output_folder_path = "./data/mixed_recordings/normalised_recordings/visc_normalized_20dB/"
for i in trange(len(input_paths)):
    audio_normalizer(input_paths[i], output_folder_path,file_names[i], -20.0)

  4%|▎         | 213/5980 [00:04<01:44, 55.29it/s]c:\Users\Eryk\anaconda3\envs\Magisterka\Lib\site-packages\pyloudnorm\normalize.py:58: RuntimeWarning: invalid value encountered in multiply
  output = gain * data
100%|██████████| 5980/5980 [01:23<00:00, 71.73it/s]


In [39]:
# Assuming df_train_filtered and df_visc are your dataframes

# Add a random column to df_visc for shuffling
df_visc['random_column'] = np.random.rand(len(df_visc))

# Reset the index of both dataframes for easy concatenation later
df_train_filterred.reset_index(drop=True, inplace=True)
df_visc.reset_index(drop=True, inplace=True)

# Determine the number of repetitions needed for each row in df_visc
repetitions = len(df_train_filterred) // len(df_visc) + 1

# Repeat rows from df_visc
df_visc_repeated = pd.concat([df_visc]*repetitions, ignore_index=True)

# Shuffle the rows of df_visc_repeated
df_visc_repeated = df_visc_repeated.sample(frac=1).reset_index(drop=True)

# Take only the required number of rows from df_visc_repeated
df_visc_repeated = df_visc_repeated.iloc[:len(df_train_filterred)]

# Combine the two dataframes
merged_df = pd.concat([df_train_filterred, df_visc_repeated], axis=1)

# Drop the random column from df_visc
df_visc.drop(columns='random_column', inplace=True)


In [40]:
merged_df

,index,audioname,ref_orig,audiopath_bigos,audiopath_local,audiopath_normalized,file_names,file_path,class_id,class,random_column
0,0,pjatk-clarin_mobile-15-train-0035-00001,proszę państwa senatorów o zajęcie miejsc przy...,pjatk-clarin_mobile-15-train-0035-00001.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,1 (211).wav,.\data\VISC Dataset SON\1 (211).wav,1,bus_interior,0.370624
1,1,pjatk-clarin_mobile-15-train-0035-00002,proszę nowo wybranego senatora stanisława husk...,pjatk-clarin_mobile-15-train-0035-00002.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,8 (533).wav,.\data\VISC Dataset SON\8 (533).wav,8,other_car_interior,0.014508
2,2,pjatk-clarin_mobile-15-train-0035-00003,sejm przyjął większość poprawek senatu do usta...,pjatk-clarin_mobile-15-train-0035-00003.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,6 (590).wav,.\data\VISC Dataset SON\6 (590).wav,6,truck_interior,0.983117
3,3,pjatk-clarin_mobile-15-train-0035-00004,pomimo faktu że w tęczy występuje niemal ciągł...,pjatk-clarin_mobile-15-train-0035-00004.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,3 (296).wav,.\data\VISC Dataset SON\3 (296).wav,3,pickup_interior,0.637204
4,4,pjatk-clarin_mobile-15-train-0035-00005,był autorem pierwszego w historii polskiego op...,pjatk-clarin_mobile-15-train-0035-00005.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,8 (302).wav,.\data\VISC Dataset SON\8 (302).wav,8,other_car_interior,0.382808
...,...,...,...,...,...,...,...,...,...,...,...
79520,82020,polyai-minds14-21-train-0001-00458,Czy mogę przelać pieniądze na konto,polyai-minds14-21-train-0001-00458.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,1 (144).wav,.\data\VISC Dataset SON\1 (144).wav,1,bus_interior,0.749818
79521,82021,polyai-minds14-21-train-0001-00459,jestem bardzo zatyczki serdecznie Mam problem ...,polyai-minds14-21-train-0001-00459.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,1 (575).wav,.\data\VISC Dataset SON\1 (575).wav,1,bus_interior,0.386108
79522,82022,polyai-minds14-21-train-0001-00460,Dzień dobry Chciałbym wpłacić pieniądze na swo...,polyai-minds14-21-train-0001-00460.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,3 (214).wav,.\data\VISC Dataset SON\3 (214).wav,3,pickup_interior,0.111998
79523,82023,polyai-minds14-21-train-0001-00461,Chcę wpłacić pieniądze na swoje konto w jakiś ...,polyai-minds14-21-train-0001-00461.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,8 (692).wav,.\data\VISC Dataset SON\8 (692).wav,8,other_car_interior,0.216880


In [42]:
merged_df['noise_normalised'] = "./data/mixed_recordings/normalised_recordings/visc_normalized_20dB/" + merged_df['file_names']

In [37]:
snr_values = [25,10]

In [45]:

# Loop through each SNR value
for snr in snr_values:
    # Create a folder for the current SNR value
    folder_path = f'./data/mixed_recordings/SNR_{snr}'
    os.makedirs(folder_path, exist_ok=True)
    # Loop through the dataframe and mix files for the current SNR value
    snr_paths = []
    for index, row in merged_df.iterrows():
        signal_path = row['audiopath_normalized']
        noise_path = row['noise_normalised']
        audio_name = row['audiopath_bigos']
        save_path = os.path.join(folder_path, audio_name)  # Change the naming convention if needed
        snr_paths.append(save_path)

        # Call your mixer function here
        mixer(signal_path, noise_path, snr, save_path)
    
    column_name = f"Mixed_audio_SNR_{snr}_path"
    merged_df[column_name] = snr_paths

In [47]:
#merged_df.to_parquet('./data/parquets/training_file.parquet.gzip', compression= 'gzip')

## Traning

In [25]:
training_df = pd.read_parquet('./data/parquets/training_file.parquet.gzip')

In [4]:
training_df

,index,audioname,ref_orig,audiopath_bigos,audiopath_local,audiopath_normalized,file_names,file_path,class_id,class,random_column,noise_normalised,Mixed_audio_SNR_25_path,Mixed_audio_SNR_10_path
0,0,pjatk-clarin_mobile-15-train-0035-00001,proszę państwa senatorów o zajęcie miejsc przy...,pjatk-clarin_mobile-15-train-0035-00001.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,1 (211).wav,.\data\VISC Dataset SON\1 (211).wav,1,bus_interior,0.370624,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/SNR_25\pjatk-clarin_mo...,./data/mixed_recordings/SNR_10\pjatk-clarin_mo...
1,1,pjatk-clarin_mobile-15-train-0035-00002,proszę nowo wybranego senatora stanisława husk...,pjatk-clarin_mobile-15-train-0035-00002.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,8 (533).wav,.\data\VISC Dataset SON\8 (533).wav,8,other_car_interior,0.014508,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/SNR_25\pjatk-clarin_mo...,./data/mixed_recordings/SNR_10\pjatk-clarin_mo...
2,2,pjatk-clarin_mobile-15-train-0035-00003,sejm przyjął większość poprawek senatu do usta...,pjatk-clarin_mobile-15-train-0035-00003.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,6 (590).wav,.\data\VISC Dataset SON\6 (590).wav,6,truck_interior,0.983117,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/SNR_25\pjatk-clarin_mo...,./data/mixed_recordings/SNR_10\pjatk-clarin_mo...
3,3,pjatk-clarin_mobile-15-train-0035-00004,pomimo faktu że w tęczy występuje niemal ciągł...,pjatk-clarin_mobile-15-train-0035-00004.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,3 (296).wav,.\data\VISC Dataset SON\3 (296).wav,3,pickup_interior,0.637204,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/SNR_25\pjatk-clarin_mo...,./data/mixed_recordings/SNR_10\pjatk-clarin_mo...
4,4,pjatk-clarin_mobile-15-train-0035-00005,był autorem pierwszego w historii polskiego op...,pjatk-clarin_mobile-15-train-0035-00005.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,8 (302).wav,.\data\VISC Dataset SON\8 (302).wav,8,other_car_interior,0.382808,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/SNR_25\pjatk-clarin_mo...,./data/mixed_recordings/SNR_10\pjatk-clarin_mo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79520,82020,polyai-minds14-21-train-0001-00458,Czy mogę przelać pieniądze na konto,polyai-minds14-21-train-0001-00458.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,1 (144).wav,.\data\VISC Dataset SON\1 (144).wav,1,bus_interior,0.749818,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/SNR_25\polyai-minds14-...,./data/mixed_recordings/SNR_10\polyai-minds14-...
79521,82021,polyai-minds14-21-train-0001-00459,jestem bardzo zatyczki serdecznie Mam problem ...,polyai-minds14-21-train-0001-00459.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,1 (575).wav,.\data\VISC Dataset SON\1 (575).wav,1,bus_interior,0.386108,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/SNR_25\polyai-minds14-...,./data/mixed_recordings/SNR_10\polyai-minds14-...
79522,82022,polyai-minds14-21-train-0001-00460,Dzień dobry Chciałbym wpłacić pieniądze na swo...,polyai-minds14-21-train-0001-00460.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/mixed_recordings/normalised_recordings/...,3 (214).wav,.\data\VISC Dataset SON\3 (214).wav,3,pickup_interior,0.111998,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/SNR_25\polyai-minds14-...,./data/mixed_recordings/SNR_10\polyai-minds14-...
79523,82023,polyai-minds14-21-train-0001-00461,Chcę wpłacić pienią

In [26]:
train_df, test_df = train_test_split(creating_random_split_df(training_df,750), test_size=0.2, random_state=42)

In [27]:
training_dataset = Dataset.from_dict({"audio": train_df['Mixed_audio_SNR_25_path'], "sentences":train_df['ref_orig']}).cast_column("audio", Audio(sampling_rate=16000))
testing_dataset = Dataset.from_dict({"audio": test_df['Mixed_audio_SNR_25_path'], "sentences":test_df['ref_orig']}).cast_column("audio", Audio(sampling_rate=16000))


In [28]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-medium")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

c:\Users\Eryk\anaconda3\envs\Magisterka\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Eryk\.cache\huggingface\hub\models--openai--whisper-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [29]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-medium", language="polish", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Small test

In [30]:
input_str = testing_dataset["sentences"][0]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Co ma to wspólnego ze sprawiedliwością?
Decoded w/ special:    <|startoftranscript|><|pl|><|transcribe|><|notimestamps|>Co ma to wspólnego ze sprawiedliwością?<|endoftext|>
Decoded w/out special: Co ma to wspólnego ze sprawiedliwością?
Are equal:             True


In [31]:
processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="polish", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [32]:
training_dataset[0]

{'audio': {'path': './data/mixed_recordings/SNR_25\\mozilla-common_voice_15-23-train-2856-01693.wav',
  'array': array([-0.00095098, -0.00161573, -0.00225208, ..., -0.00331347,
         -0.00230933, -0.00110459]),
  'sampling_rate': 16000},
 'sentences': 'To bezcelowe'}

In [33]:
import torch

In [34]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentences"]).input_ids
    return batch

In [35]:
del training_df
del train_df
del test_df

In [36]:
training_dataset = training_dataset.map(prepare_dataset)
testing_dataset = testing_dataset.map(prepare_dataset)

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [37]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [38]:
torch.cuda.is_available()

True

In [39]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [40]:
import evaluate

metric = evaluate.load("wer")

In [41]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [42]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

In [43]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [24]:
torch.cuda.empty_cache()

In [47]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-med-pol-car",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    generation_max_length=150,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [48]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=training_dataset,
    eval_dataset=testing_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [49]:
trainer.train()

  0%|          | 0/4000 [00:00<?, ?it/s]

{'loss': 1.506, 'learning_rate': 4.2000000000000006e-07, 'epoch': 0.67}
{'loss': 1.4442, 'learning_rate': 9.200000000000001e-07, 'epoch': 1.33}
{'loss': 1.1312, 'learning_rate': 1.42e-06, 'epoch': 2.0}
{'loss': 0.8122, 'learning_rate': 1.9200000000000003e-06, 'epoch': 2.67}
{'loss': 0.6313, 'learning_rate': 2.42e-06, 'epoch': 3.33}
{'loss': 0.3258, 'learning_rate': 2.92e-06, 'epoch': 4.0}
{'loss': 0.1346, 'learning_rate': 3.4200000000000007e-06, 'epoch': 4.67}
{'loss': 0.0935, 'learning_rate': 3.920000000000001e-06, 'epoch': 5.33}
{'loss': 0.068, 'learning_rate': 4.42e-06, 'epoch': 6.0}
{'loss': 0.0306, 'learning_rate': 4.92e-06, 'epoch': 6.67}
{'loss': 0.0279, 'learning_rate': 5.420000000000001e-06, 'epoch': 7.33}
{'loss': 0.0174, 'learning_rate': 5.92e-06, 'epoch': 8.0}
{'loss': 0.0099, 'learning_rate': 6.42e-06, 'epoch': 8.67}
{'loss': 0.0084, 'learning_rate': 6.92e-06, 'epoch': 9.33}
{'loss': 0.0077, 'learning_rate': 7.420000000000001e-06, 'epoch': 10.0}
{'loss': 0.005, 'learning_r

  0%|          | 0/75 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


{'eval_loss': 0.3735416531562805, 'eval_wer': 100.0, 'eval_runtime': 126.3904, 'eval_samples_per_second': 1.187, 'eval_steps_per_second': 0.593, 'epoch': 26.67}


...

  0%|          | 0/75 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


{'eval_loss': 0.4174261689186096, 'eval_wer': 100.06523157208089, 'eval_runtime': 126.1714, 'eval_samples_per_second': 1.189, 'eval_steps_per_second': 0.594, 'epoch': 53.33}


c:\Users\Eryk\anaconda3\envs\Magisterka\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.0002, 'learning_rate': 5.654285714285714e-06, 'epoch': 54.0}
{'loss': 0.0002, 'learning_rate': 5.582857142857143e-06, 'epoch': 54.67}
{'loss': 0.0002, 'learning_rate': 5.511428571428572e-06, 'epoch': 55.33}
{'loss': 0.0002, 'learning_rate': 5.4400000000000004e-06, 'epoch': 56.0}
{'loss': 0.0001, 'learning_rate': 5.368571428571429e-06, 'epoch': 56.67}
{'loss': 0.0001, 'learning_rate': 5.297142857142858e-06, 'epoch': 57.33}
{'loss': 0.0001, 'learning_rate': 5.225714285714286e-06, 'epoch': 58.0}
{'loss': 0.0001, 'learning_rate': 5.154285714285715e-06, 'epoch': 58.67}
{'loss': 0.0001, 'learning_rate': 5.082857142857144e-06, 'epoch': 59.33}
{'loss': 0.0001, 'learning_rate': 5.011428571428571e-06, 'epoch': 60.0}
{'loss': 0.0001, 'learning_rate': 4.94e-06, 'epoch': 60.67}
{'loss': 0.0001, 'learning_rate': 4.868571428571429e-06, 'epoch': 61.33}
{'loss': 0.0001, 'learning_rate': 4.797142857142857e-06, 'epoch': 62.0}
{'loss': 0.0001, 'learning_rate': 4.725714285714286e-06, 'epoch': 62